<a href="https://colab.research.google.com/github/dauphineezc/NLP-Final-Project/blob/main/NLPfinalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>